In [67]:
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
import warnings
warnings.filterwarnings('ignore')

In [62]:
pdf = "/content/resume.pdf"
pdf_reader = PdfReader(pdf)
print(pdf_reader)

In [63]:
# extrat text from each page separately
text = ""
for page in pdf_reader.pages:
    text += page.extract_text()

print(text)

GOPINATH ASOKAN 
Data Science Enthusiast 
Passionate data science enthusiast with a strong foundation in diverse industries. Equipped with 5+ years
of industry experience, highly skilled in problem-solving, and project management. Eager to seamlessly
merge analytical skills with artistic expertise for impactful insights and innovation. Excited to apply data-
driven strategies to challenges, contribute proactively and eﬀectively to the ﬁeld, and drive innovation. 
gopiashokankiot@gmail.com 
linkedin.com/in/gopiashokan 
github.com/gopiashokan 
WORK EXPERIENCE 
Senior Process Executive - Operations 
Mahendra Next Wealth IT India Pvt Ltd 
05/2019 - 12/2022
, 
 
Namakkal 
Proﬁciently executed image editing tasks for bigbasket's
product images, encompassing renaming, retouching, 
color
correction, content cropping, and photo manipulation. 
Expertly designed captivating banners and creatives for
advertisements, skillfully integrating combo packs, multi-
packs, and hero images into Bigbasket's

In [64]:
# Split the long text into small chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=700,
                                               chunk_overlap=200,
                                               length_function=len)

chunks = text_splitter.split_text(text=text)
chunks

['GOPINATH ASOKAN \nData Science Enthusiast \nPassionate data science enthusiast with a strong foundation in diverse industries. Equipped with 5+ years\nof industry experience, highly skilled in problem-solving, and project management. Eager to seamlessly\nmerge analytical skills with artistic expertise for impactful insights and innovation. Excited to apply data-\ndriven strategies to challenges, contribute proactively and eﬀectively to the ﬁeld, and drive innovation. \ngopiashokankiot@gmail.com \nlinkedin.com/in/gopiashokan \ngithub.com/gopiashokan \nWORK EXPERIENCE \nSenior Process Executive - Operations \nMahendra Next Wealth IT India Pvt Ltd \n05/2019 - 12/2022\n, \n \nNamakkal',
 "linkedin.com/in/gopiashokan \ngithub.com/gopiashokan \nWORK EXPERIENCE \nSenior Process Executive - Operations \nMahendra Next Wealth IT India Pvt Ltd \n05/2019 - 12/2022\n, \n \nNamakkal \nProﬁciently executed image editing tasks for bigbasket's\nproduct images, encompassing renaming, retouching, \ncol

In [65]:
chunks[0]

'GOPINATH ASOKAN \nData Science Enthusiast \nPassionate data science enthusiast with a strong foundation in diverse industries. Equipped with 5+ years\nof industry experience, highly skilled in problem-solving, and project management. Eager to seamlessly\nmerge analytical skills with artistic expertise for impactful insights and innovation. Excited to apply data-\ndriven strategies to challenges, contribute proactively and eﬀectively to the ﬁeld, and drive innovation. \ngopiashokankiot@gmail.com \nlinkedin.com/in/gopiashokan \ngithub.com/gopiashokan \nWORK EXPERIENCE \nSenior Process Executive - Operations \nMahendra Next Wealth IT India Pvt Ltd \n05/2019 - 12/2022\n, \n \nNamakkal'

In [66]:
chunks[1]

"linkedin.com/in/gopiashokan \ngithub.com/gopiashokan \nWORK EXPERIENCE \nSenior Process Executive - Operations \nMahendra Next Wealth IT India Pvt Ltd \n05/2019 - 12/2022\n, \n \nNamakkal \nProﬁciently executed image editing tasks for bigbasket's\nproduct images, encompassing renaming, retouching, \ncolor\ncorrection, content cropping, and photo manipulation. \nExpertly designed captivating banners and creatives for\nadvertisements, skillfully integrating combo packs, multi-\npacks, and hero images into Bigbasket's product pages. \nContributed to taxonomy by mapping tax codes, manually\nidentiﬁed competitor products, and veriﬁed AI-generated\noutputs for accuracy, assisting in AI improvement eﬀorts."

"linkedin.com/in/gopiashokan \ngithub.com/gopiashokan \nWORK EXPERIENCE \nSenior Process Executive - Operations \nMahendra Next Wealth IT India Pvt Ltd \n05/2019 - 12/2022\n, \n \nNamakkal"

The above text is common(overlap) for both chunks[0] and chunks[1].
(chunk_overlap=200 - maximum length, it means length is not exceed 200)

In [7]:
openai_api_key = input('Enter you OpenAI API Key: ')

In [52]:
def openai(openai_api_key, chunks, analyze):

    # Using OpenAI service for embedding
    embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

    # Facebook AI Similarity Serach library help us to convert text data to numerical vector
    vectorstores = FAISS.from_texts(chunks, embedding=embeddings)

    # compares the query and chunks, enabling the selection of the top 'K' most similar chunks based on their similarity scores.
    docs = vectorstores.similarity_search(query=analyze, k=3)

    # creates an OpenAI object, using the ChatGPT 3.5 Turbo model
    llm = ChatOpenAI(model='gpt-3.5-turbo', api_key=openai_api_key)

    # question-answering (QA) pipeline, making use of the load_qa_chain function
    chain = load_qa_chain(llm=llm, chain_type='stuff')

    response = chain.run(input_documents=docs, question=analyze)
    return response

In [55]:
def resume_summary(query_with_chunks):
    query = f''' need to detailed summarization of below resume and finally conclude them

                """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
                {query_with_chunks}
                """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
                '''
    return query

summary = resume_summary(query_with_chunks=chunks)
summary_result = openai(openai_api_key=openai_api_key, chunks=chunks, analyze=summary)
print(summary_result)

The resume belongs to Gopinath Asokan, who is a data science enthusiast with a strong foundation in diverse industries. He has 5+ years of industry experience and is highly skilled in problem-solving and project management. Gopinath is eager to merge his analytical skills with artistic expertise for impactful insights and innovation. He is excited to apply data-driven strategies to challenges and contribute proactively to the field. 

In terms of work experience, Gopinath has worked as a Senior Process Executive - Operations at Mahendra Next Wealth IT India Pvt Ltd from 05/2019 to 12/2022. He was responsible for precise resume analysis and suggestions, as well as integrating Selenium for dynamic LinkedIn data extraction. He also implemented machine learning for precise retail sales predictions, emphasizing preprocessing and algorithm selection.

Gopinath has also worked as an Associate Engineer - Quality at Rudra Blades and Edges Pvt Ltd from 07/2018 to 12/2018. He performed continuous

In [56]:
def resume_strength(query_with_chunks):
    query = f'''need to detailed analysis and explain of the strength of below resume and finally conclude them
                """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
                {query_with_chunks}
                """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
                '''
    return query

strength = resume_strength(query_with_chunks=summary_result)
strength_result = openai(openai_api_key=openai_api_key, chunks=chunks, analyze=strength)
print(strength_result)

Strengths of Gopinath Asokan's resume:

1. Strong foundation in diverse industries: Gopinath's resume highlights his experience and expertise in various industries, showcasing his adaptability and ability to work in different environments.

2. 5+ years of industry experience: Gopinath's extensive experience in the industry demonstrates his ability to handle real-world challenges and shows his level of expertise in the field.

3. Strong problem-solving and project management skills: Gopinath's resume emphasizes his skills in problem-solving and project management, which are crucial in the field of data science. This indicates his ability to effectively handle complex problems and successfully manage projects.

4. Analytical skills combined with artistic expertise: Gopinath's resume mentions his eagerness to merge his analytical skills with artistic expertise, indicating his ability to think creatively and produce impactful insights and innovations.

5. Data-driven strategies: Gopinath's

In [60]:
def resume_weakness(query_with_chunks):
    query = f'''need to detailed analysis and explain of the weakness of below resume and how to improve make a better resume.

                """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
                {query_with_chunks}
                """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
                '''
    return query

weakness = resume_weakness(query_with_chunks=summary_result)
result_weakness = openai(openai_api_key=openai_api_key, chunks=chunks, analyze=weakness)
print(result_weakness)

Based on the provided resume details, Gopinath Asokan has a strong foundation in diverse industries and is highly skilled in problem-solving and project management. However, there are a few weaknesses in the resume that can be improved:

1. Lack of a clear career objective: The resume does not mention a specific career objective or goal. It would be beneficial to include a clear and concise objective statement that highlights Gopinath's career aspirations and how his skills and experience align with those goals.

2. Incomplete work experience details: While the resume mentions Gopinath's job titles and responsibilities, it does not provide specific accomplishments or achievements in each role. Adding quantifiable achievements or results-oriented statements would strengthen the resume and demonstrate Gopinath's impact in previous positions.

3. Limited information on education: The resume briefly mentions Gopinath's educational background, but it lacks details on specific coursework or 

In [61]:
def job_title_suggestion(query_with_chunks):

    query = f''' what are the job roles i apply to likedin based on below?
                  
                  """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
                  {query_with_chunks}
                  """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
                '''
    return query

suggestion = job_title_suggestion(query_with_chunks=summary_result)
result_suggestion = openai(openai_api_key=openai_api_key, chunks=chunks, analyze=suggestion)
print(result_suggestion)

Based on the information provided, some potential job roles that Gopinath Asokan could apply to on LinkedIn include:

1. Data Scientist
2. Data Analyst
3. Machine Learning Engineer
4. Business Analyst
5. Project Manager
6. Operations Analyst
7. Quality Assurance Engineer
8. Sales Analyst
9. AI Engineer
10. Retail Analyst

These job roles align with Gopinath's skills and experience in data science, problem-solving, project management, resume analysis, retail sales forecasting, and quality assurance.
